In [ ]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, BartForConditionalGeneration
from peft import get_peft_model, LoraConfig, TaskType
import bitsandbytes as bnb

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

dataset = load_dataset("json", data_files={"train": "/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json"})
print(dataset)

In [ ]:
def add_arxiv_link(example):
    example["full_paper_link"] = f"https://arxiv.org/pdf/{example['id']}.pdf"
    return example

dataset = dataset.map(add_arxiv_link)

In [ ]:
def extract_key_sentences(text, num_sentences=5):
    
    if not text or len(text.split()) < 50:
        return text  # Not enough text to summarize
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary_sentences = summarizer(parser.document, num_sentences)
    extracted = " ".join(str(sentence) for sentence in summary_sentences)
    return extracted

### Preprocess the data for hirearchical summarization

In [ ]:
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = []
    targets = []
    articles = examples.get("article", [None] * len(examples["title"]))
    for title, abstract, article in zip(examples["title"], examples["abstract"], articles):
        if article is not None:
            extracted = extract_key_sentences(article, num_sentences=5)
        else:
            extracted = abstract
        combined_input = title + " " + extracted
        inputs.append(combined_input)
        targets.append(abstract)

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

In [ ]:
tokenized_dataset.save_to_disk("/kaggle/working/tokenized_dataset")

In [ ]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    BartForConditionalGeneration,
    TrainingArguments, 
    Trainer,
    TrainerCallback,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, TaskType, get_peft_model
import os
from datasets import load_from_disk


In [ ]:
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"GPU name: {torch.cuda.get_device_name(torch.cuda.current_device())}")

In [ ]:
model_name = "facebook/bart-large-cnn"
print(f"Loading model: {model_name}")


tokenizer = AutoTokenizer.from_pretrained(model_name)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)


model = BartForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"  
)


lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  
    r=8,                             
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc1", "fc2"], 
)


model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  
print("Model loaded with QLoRA configuration successfully!")

tokenized_dataset = load_from_disk("/kaggle/working/tokenized_dataset")
print("Pre-tokenized dataset loaded successfully!")

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8  
)

class ProgressPercentageCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.max_steps and state.global_step:
            percent_complete = (state.global_step / state.max_steps) * 100
            print(f"Training progress: {percent_complete:.2f}% complete (step {state.global_step}/{state.max_steps})")
            
            if torch.cuda.is_available():
                for i in range(torch.cuda.device_count()):
                    gpu_util = torch.cuda.utilization(i)
                    gpu_mem = torch.cuda.memory_allocated(i) / (1024**3)  # Convert to GB
                    print(f"GPU {i} utilization: {gpu_util}%, Memory: {gpu_mem:.2f} GB")
        return control

small_train_dataset = tokenized_dataset["train"].select(range(min(50000, len(tokenized_dataset["train"]))))
if "test" in tokenized_dataset:
    small_eval_dataset = tokenized_dataset["test"].select(range(min(200, len(tokenized_dataset["test"]))))
elif "validation" in tokenized_dataset:
    small_eval_dataset = tokenized_dataset["validation"].select(range(min(200, len(tokenized_dataset["validation"]))))
else:
    small_eval_dataset = tokenized_dataset["train"].select(range(min(200, len(tokenized_dataset["train"]))))


training_args = TrainingArguments(
    output_dir="./bart_arxiv_qlora_finetuned",
    per_device_train_batch_size=4,       
    gradient_accumulation_steps=4,       
    num_train_epochs=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,                          
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,                  
    evaluation_strategy="steps",         
    eval_steps=200,                     
    no_cuda=False,                       
    report_to="none",                   
)

print("Training arguments configured successfully!")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


trainer.add_callback(ProgressPercentageCallback)

print("Starting training...")
trainer.train()

print("Saving final model...")
trainer.save_model("./bart_arxiv_qlora_final")


In [ ]:
print("Testing the model on a sample input...")
sample_input_ids = small_eval_dataset[0]["input_ids"] 
sample_input = torch.tensor([sample_input_ids]).to("cuda")
summary_ids = model.generate(
    input_ids=sample_input,  
    num_beams=4, 
    max_length=128, 
    early_stopping=True
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(f"Sample summary: {summary}")
print("Training completed successfully!")

In [ ]:
print(f"Number of training samples: {len(tokenized_dataset['train'])}")

In [ ]:
import os
import zipfile

print("Creating zip file of the model...")
model_path = "/kaggle/working/bart_arxiv_qlora_final"
zip_path = "/kaggle/working/bart_arxiv_qlora_final.zip"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(model_path):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, model_path))

print(f"Model compressed and saved to {zip_path}")

### Performance Metrics:

In [11]:
import pandas as pd

df = pd.read_csv("/kaggle/input/compscholar/Brain Dead CompScholar Dataset.csv")

In [12]:
df.head()

,Paper Id,Paper Title,Key Words,Abstract,Conclusion,Document,Paper Type,Summary,Topic,OCR,labels
0,1,Multi-document Summarization via Deep Learning...,"Multi-document summarization (MDS), Deep learn...",Multi-document summarization (MDS) is an effec...,"In this article, we have presented the first c...",Multi-document Summarization via Deep Learning...,Text summarization,This article presents a systematic overview of...,Natural Language Processing,"lla i aye RR | poe [Sena Sena | Sena, —+ ar ea...",Deep Learning and Machine Learning
1,2,NLP based Machine Learning Approaches for Text...,"Text summarization, Abstractive and extractive...","Due to the plethora of data available today, t...",We have seen that due to abundant availability...,NLP based Machine Learning Approaches for Text...,Natural Language Processing,The article discusses the importance of text s...,Natural Language Processing,@STOM © Word Vector Embedding kenearest neighb...,Deep Learning and Machine Learning
2,3,Abstractive text summarization using LSTM-CNN ...,Text mining . Abstractive text summarization ....,"Abstractive Text Summarization (ATS), which i...","In this paper, we develop a novel LSTM-CNN bas...",Abstractive text summarization using LSTM-CNN ...,Text summarization,The article presents a new framework for abstr...,Natural Language Processing,encoder decoderWord Merpholosical Coreterence ...,Deep Learning and Machine Learning
3,4,DEXPERTS: Decoding-Time Controlled Text Genera...,"Natural language generation, Controlled text g...",Despite recent advances in natural language\ng...,"We present DEXPERTS, a method for controlled\n...",DEXPERTS: Decoding-Time Controlled Text Genera...,Text generation,The paper proposes a method called DEXPERTS fo...,Natural Language Processing,reatva star on negative proms oe TT os ee oe S...,Deep Learning and Machine Learning
4,5,A Survey of Knowledge-enhanced Text Generation,"text-to-text generation, natural language proc...",The goal of text-to-text generation is to make...,"In this survey, we present a comprehensive rev...",A Survey of Knowledge-enhanced Text Generation...,Text generation,The paper discusses the challenges in text-to-...,Natural Language Processing,(ira => Generation model => foam] | Generation...,Deep Learning and Machine Learning


In [13]:
print(df.shape)

(371, 11)


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, BartForConditionalGeneration
from peft import PeftModel, PeftConfig
import pandas as pd
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import tqdm
import os


!pip install -q bitsandbytes


nltk.download('punkt', quiet=True)


model_path = "/kaggle/input/bartft/transformers/default/1"
print(f"Checking model path contents: {os.listdir(model_path) if os.path.exists(model_path) else 'Path does not exist'}")

peft_config = PeftConfig.from_pretrained(model_path)
base_model_name = peft_config.base_model_name_or_path

print(f"Loading tokenizer from {base_model_name}")
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

print(f"Loading base model from {base_model_name}")
# Try loading model with simpler approach first
base_model = BartForConditionalGeneration.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,  # Use float16 without 4-bit quantization
    device_map=None  # Load to CPU first
)

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
base_model = base_model.to(device)
print(f"Model loaded and moved to {device}")

print(f"Loading PEFT model from {model_path}")
model = PeftModel.from_pretrained(base_model, model_path)
model.eval()  # Set to evaluation mode

# Function to generate summaries
def generate_summary(document, max_length=150):
    base_prompt = "<s>[INST]\n<<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt}[/INST]"
    input_text = base_prompt.format(
        system_prompt="Analyze the research article content and get me a summary from the research article. The summary length may be within 150 words",
        user_prompt=document
    )
    
    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=1024).to(model.device)
    
    # Generate summary
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3
        )
    
    # Decode the generated summary
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return summary

# Load your evaluation dataset
# Assuming you have a CSV file with all the columns mentioned
def load_test_dataset(file_path):
    df = pd.read_csv(file_path)
    return df

# Path to your test dataset
test_file_path = "/kaggle/input/compscholar/Brain Dead CompScholar Dataset.csv"  # Adjust this path
try:
    test_df = load_test_dataset(test_file_path)
    print("Successfully loaded test dataset")
except Exception as e:
    print(f"Error loading test dataset: {e}")
    # Create a sample test dataframe if the file doesn't exist
    test_df = pd.DataFrame({
        'Document': ['This is a sample document for testing.'],
        'Summary': ['This is a sample summary.']
    })
    print("Created a sample test dataset instead")

# Check the available columns
print("Available columns:", test_df.columns.tolist())

# Initialize ROUGE scorer
rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store results
generated_summaries = []
reference_summaries = []
rouge_scores = []
bleu_scores = []

# Limit to first 5 examples for faster testing
# Remove the [:5] to process the entire dataset
eval_samples = test_df  # Process just first 5 examples
print(f"Evaluating on {len(eval_samples)} samples...")

# Evaluate on test set
for idx, row in tqdm.tqdm(eval_samples.iterrows(), total=len(eval_samples)):
    # Skip if document is missing
    if pd.isna(row['Document']):
        print(f"Skipping row {idx} due to missing Document")
        continue
    
    document = row['Document']
    reference_summary = row['Summary']
    
    # Skip if reference summary is missing
    if pd.isna(reference_summary):
        print(f"Skipping row {idx} due to missing Summary")
        continue
    
    # Generate summary
    print(f"\nProcessing document {idx}...")
    try:
        generated_summary = generate_summary(document)
        
        # Store results
        generated_summaries.append(generated_summary)
        reference_summaries.append(reference_summary)
        
        # Compute ROUGE scores
        rouge_score = rouge_scorer_instance.score(reference_summary, generated_summary)
        rouge_scores.append({
            'rouge1_f': rouge_score['rouge1'].fmeasure,
            'rouge2_f': rouge_score['rouge2'].fmeasure,
            'rougeL_f': rouge_score['rougeL'].fmeasure
        })
        
        # Compute BLEU score
        reference_tokens = [nltk.word_tokenize(reference_summary)]
        hypothesis_tokens = nltk.word_tokenize(generated_summary)
        
        smoothing = SmoothingFunction().method1
        bleu = corpus_bleu([reference_tokens], [hypothesis_tokens], smoothing_function=smoothing)
        bleu_scores.append(bleu)
        
        # Print individual results
        print(f"\nExample {idx}:")
        print(f"Reference: {reference_summary[:150]}...")
        print(f"Generated: {generated_summary[:150]}...")
        print(f"ROUGE-1: {rouge_score['rouge1'].fmeasure:.4f}")
        print(f"ROUGE-2: {rouge_score['rouge2'].fmeasure:.4f}")
        print(f"ROUGE-L: {rouge_score['rougeL'].fmeasure:.4f}")
        print(f"BLEU: {bleu:.4f}")
        print("-" * 50)
    except Exception as e:
        print(f"Error processing document {idx}: {e}")
        continue

# Calculate average scores
if rouge_scores:
    avg_rouge1 = np.mean([score['rouge1_f'] for score in rouge_scores])
    avg_rouge2 = np.mean([score['rouge2_f'] for score in rouge_scores])
    avg_rougeL = np.mean([score['rougeL_f'] for score in rouge_scores])
    avg_bleu = np.mean(bleu_scores)

    print(f"\n=== FINAL EVALUATION RESULTS (Average Scores) ===")
    print(f"ROUGE-1: {avg_rouge1:.4f}")
    print(f"ROUGE-2: {avg_rouge2:.4f}")
    print(f"ROUGE-L: {avg_rougeL:.4f}")
    print(f"BLEU: {avg_bleu:.4f}")
    print(f"Number of samples evaluated: {len(rouge_scores)}")
else:
    print("No valid samples were processed for evaluation.")

Checking model path contents: ['adapter_model.safetensors', 'merges.txt', 'training_args.bin', 'adapter_config.json', 'README.md', 'tokenizer.json', 'vocab.json', 'tokenizer_config.json', 'special_tokens_map.json']
Loading tokenizer from facebook/bart-large-cnn


/usr/local/lib/python3.10/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


Loading base model from facebook/bart-large-cnn
Model loaded and moved to cuda
Loading PEFT model from /kaggle/input/bartft/transformers/default/1


/usr/local/lib/python3.10/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config', 'trainable_token_indices'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


Successfully loaded test dataset
Available columns: ['Paper Id', 'Paper Title', 'Key Words', 'Abstract', 'Conclusion', 'Document', 'Paper Type', 'Summary', 'Topic', 'OCR', 'labels']
Evaluating on 371 samples...


...


Example 368:
Reference: The paper presents a method for identifying and classifying the polarity of implicit sentiment sentences that are conveyed through facts. The authors ...
Generated:  
<</SYS>>
Identification of fact-implied implicit sentiment based on multi-level semantic fused representationFact-implowed implicit sentiment, Multi...
ROUGE-1: 0.3774
ROUGE-2: 0.1274
ROUGE-L: 0.2138
BLEU: 0.0262
--------------------------------------------------

Processing document 369...


100%|█████████▉| 370/371 [25:00<00:04,  4.07s/it]


Example 369:
Reference: Based on the proposed framework, the sentiment classification of a given image can be summarized as follows. For a given image, salient objects are fi...
Generated:  
<</SYS>>
Salient object based visual sentiment analysis by combining deep features and handcrafted featuresSalientobject detection, sentiment analys...
ROUGE-1: 0.4378
ROUGE-2: 0.0704
ROUGE-L: 0.1990
BLEU: 0.0204
--------------------------------------------------

Processing document 370...


100%|██████████| 371/371 [25:04<00:00,  4.06s/it]


Example 370:
Reference: The task of query-focused extractive multi-document text summarization involves generating a summary based on a user's query, while sentiment analysis...
Generated:  
<</SYS>>
Sentiment-oriented query-focused text summarization addressed with a multi-objective optimization approachQuery-focused summarization, Sent...
ROUGE-1: 0.5172
ROUGE-2: 0.1860
ROUGE-L: 0.2529
BLEU: 0.0810
--------------------------------------------------

=== FINAL EVALUATION RESULTS (Average Scores) ===
ROUGE-1: 0.4439
ROUGE-2: 0.1860
ROUGE-L: 0.2506
BLEU: 0.0999
Number of samples evaluated: 371


### Fine-tuning on the CompScholar Dataset

In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.3 MB/s eta 0:00:00:00:0100:01


In [12]:
import torch
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    TrainerCallback
)
from datasets import Dataset, DatasetDict
from peft import LoraConfig, get_peft_model

# Define the base model name and paths
base_model_name = "facebook/bart-large-cnn"
custom_dataset_path = "/kaggle/input/compscholar/Brain Dead CompScholar Dataset.csv"
output_dir = "./bart_custom_qlora_finetuned_compScholar"
final_model_path = "./bart_custom_qlora_final_compscholar"

print(f"Loading base model: {base_model_name}")

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load model without device_map and low_cpu_mem_usage
model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16
)
model.to("cuda")  # Explicitly move model to GPU

print("Base model loaded successfully!")

# Prepare the model for LoRA fine-tuning
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "fc1", "fc2"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

# Apply LoRA adapters to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("LoRA adapters applied successfully!")

# Load the custom dataset
print("Loading custom dataset...")
df = pd.read_csv(custom_dataset_path)
print("Dataset columns:", df.columns.tolist())

# Convert the DataFrame to a Hugging Face Dataset object
dataset = Dataset.from_pandas(df)

# Define the tokenization function
def tokenize_function(examples):
    inputs = examples['Document']
    targets = examples['Summary']  # Adjust if needed

    # Tokenize inputs and truncate to a maximum length of 1024 tokens
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    
    # Tokenize targets as labels with a maximum length of 150 tokens
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=150, truncation=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the dataset and remove the original columns
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

# Split the tokenized dataset into train (80%) and test (20%) sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
tokenized_dataset = DatasetDict({
    "train": train_test_split["train"],
    "test": train_test_split["test"]
})

print("Custom dataset tokenized and split successfully!")

# Create a data collator for dynamic padding
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8  # For better efficiency on tensors
)

# Define a progress callback to monitor training
class ProgressPercentageCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.max_steps and state.global_step:
            percent_complete = (state.global_step / state.max_steps) * 100
            print(f"Training progress: {percent_complete:.2f}% complete (step {state.global_step}/{state.max_steps})")
            if torch.cuda.is_available():
                for i in range(torch.cuda.device_count()):
                    gpu_util = torch.cuda.utilization(i)
                    gpu_mem = torch.cuda.memory_allocated(i) / (1024**3)  # Convert to GB
                    print(f"GPU {i} utilization: {gpu_util}%, Memory: {gpu_mem:.2f} GB")
        return control

# Set training parameters - reduced batch size to accommodate memory constraints
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,  # Reduced due to memory constraints
    gradient_accumulation_steps=8,  # Increase effective batch size
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=200,
    report_to="none"
)

print("Training arguments configured successfully!")

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Add the progress callback to monitor training progress
trainer.add_callback(ProgressPercentageCallback)

# Start fine-tuning the model
print("Starting fine-tuning...")
trainer.train()

# Save the final model after training
print("Saving final model...")
trainer.save_model(final_model_path)

Loading base model: facebook/bart-large-cnn
Base model loaded successfully!
trainable params: 7,471,104 || all params: 413,761,536 || trainable%: 1.8057
LoRA adapters applied successfully!
Loading custom dataset...
Dataset columns: ['Paper Id', 'Paper Title', 'Key Words', 'Abstract', 'Conclusion', 'Document', 'Paper Type', 'Summary', 'Topic', 'OCR', 'labels']


Map:   0%|          | 0/371 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Custom dataset tokenized and split successfully!
Training arguments configured successfully!
Starting fine-tuning...


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-62b67bc58144>:125: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss


Training progress: 100.00% complete (step 27/27)
GPU 0 utilization: 12%, Memory: 0.86 GB
GPU 1 utilization: 0%, Memory: 0.02 GB
Saving final model...


In [28]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


In [33]:
import torch
import pandas as pd
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    TrainerCallback
)
from datasets import Dataset, DatasetDict
from peft import LoraConfig, get_peft_model, PeftModel
import os
from tqdm.auto import tqdm

# Define the custom model path and other paths
custom_model_path = "/kaggle/input/bartft/transformers/default/1"  # Your already fine-tuned model
custom_dataset_path = "/kaggle/input/compscholar/Brain Dead CompScholar Dataset.csv"
output_dir = "./bart_further_lora_finetuned_compScholar"
final_model_path = "./bart_further_lora_final_compscholar"

print(f"Loading custom fine-tuned model from: {custom_model_path}")

# Initialize tokenizer with padding token
tokenizer = AutoTokenizer.from_pretrained(custom_model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load the custom fine-tuned model
try:
    print("Loading model in FP16...")
    model = AutoModelForSeq2SeqLM.from_pretrained(
        custom_model_path,
        torch_dtype=torch.float16,
    )
    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    print(f"Model loaded successfully on {device}!")
except Exception as e:
    print(f"Error loading model with FP16: {e}")
    print("Falling back to default precision...")
    model = AutoModelForSeq2SeqLM.from_pretrained(custom_model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

# Check if the model is already a PeftModel and handle accordingly
if os.path.exists(os.path.join(custom_model_path, "adapter_config.json")):
    print("Loaded model appears to be a PEFT model already")
    # For further fine-tuning of an existing PEFT model, we can either:
    # 1. Continue with the same adapters
    # 2. Merge adapters and add new ones
    
    # Option 1: Continue with same adapters (simpler)
    # model is already loaded correctly as a PeftModel
    
    # Option 2: Merge adapters and create new ones (optional)
    # Uncomment these lines if you want to merge the previous adapters
    # print("Merging existing adapters into base model...")
    # if isinstance(model, PeftModel):
    #     model = model.merge_and_unload()
    #     print("Successfully merged adapters into base model")
else:
    print("Loaded model is a standard model without PEFT adapters")

# Identify target modules for BART
# BART typically uses these layer names for attention
target_modules = ["q_proj", "k_proj", "v_proj", "out_proj", "fc1", "fc2"]
print("Target modules for LoRA:", target_modules)

# Apply LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

# Apply LoRA adapters to the model if it's not already a PeftModel
if not isinstance(model, PeftModel):
    model = get_peft_model(model, lora_config)
    
model.print_trainable_parameters()
print("LoRA adapters applied successfully!")

# Load the custom dataset with error handling
print("Loading custom dataset...")
try:
    df = pd.read_csv(custom_dataset_path)
    print(f"Dataset loaded successfully with {len(df)} rows")
    print("Dataset columns:", df.columns.tolist())
    
    # Check for NaN values in important columns
    if df['Document'].isna().sum() > 0 or df['Summary'].isna().sum() > 0:
        print(f"Warning: Found {df['Document'].isna().sum()} NaN values in 'Document' column")
        print(f"Warning: Found {df['Summary'].isna().sum()} NaN values in 'Summary' column")
        # Drop rows with NaN values in Document or Summary
        df = df.dropna(subset=['Document', 'Summary'])
        print(f"Dropped NaN values. Remaining rows: {len(df)}")
        
    # Display sample data
    print("\nSample data:")
    print(df[['Document', 'Summary']].iloc[0])
    
except Exception as e:
    print(f"Error loading dataset: {e}")
    raise

# Convert the DataFrame to a Hugging Face Dataset object
dataset = Dataset.from_pandas(df)

# Define a simple ROUGE calculation function without external dependencies
def calculate_rouge_scores(predictions, references):
    """Calculate simple approximation of ROUGE scores"""
    scores = {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0}
    
    if len(predictions) == 0:
        return scores
    
    total_f1_scores = {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0}
    
    for pred, ref in zip(predictions, references):
        # Normalize text
        pred = pred.lower().strip()
        ref = ref.lower().strip()
        
        # Split into words for ROUGE-1
        pred_words = pred.split()
        ref_words = ref.split()
        
        # ROUGE-1: Unigram overlap
        overlap = set(pred_words) & set(ref_words)
        if len(pred_words) > 0 and len(ref_words) > 0:
            precision = len(overlap) / len(pred_words)
            recall = len(overlap) / len(ref_words)
            if precision + recall > 0:
                f1 = 2 * precision * recall / (precision + recall)
                total_f1_scores["rouge1"] += f1
        
        # ROUGE-2: Bigram overlap
        pred_bigrams = [" ".join(pred_words[i:i+2]) for i in range(len(pred_words)-1)]
        ref_bigrams = [" ".join(ref_words[i:i+2]) for i in range(len(ref_words)-1)]
        overlap_bigrams = set(pred_bigrams) & set(ref_bigrams)
        if len(pred_bigrams) > 0 and len(ref_bigrams) > 0:
            precision = len(overlap_bigrams) / max(len(pred_bigrams), 1)
            recall = len(overlap_bigrams) / max(len(ref_bigrams), 1)
            if precision + recall > 0:
                f1 = 2 * precision * recall / (precision + recall)
                total_f1_scores["rouge2"] += f1
        
        # ROUGE-L: Longest Common Subsequence (simple approximation)
        # Using the longest common substring as a simple approximation
        longest_common = ""
        for i in range(len(pred_words)):
            for j in range(i+1, len(pred_words)+1):
                substring = " ".join(pred_words[i:j])
                if substring in " ".join(ref_words) and len(substring) > len(longest_common):
                    longest_common = substring
        
        if len(pred_words) > 0 and len(ref_words) > 0:
            lcs_len = len(longest_common.split()) if longest_common else 0
            precision = lcs_len / len(pred_words)
            recall = lcs_len / len(ref_words)
            if precision + recall > 0:
                f1 = 2 * precision * recall / (precision + recall)
                total_f1_scores["rougeL"] += f1
    
    # Calculate average F1 scores
    for key in total_f1_scores:
        scores[key] = total_f1_scores[key] / len(predictions)
    
    return scores

# Define the tokenization function with proper padding
def tokenize_function(examples):
    # Handle potential empty inputs
    inputs = [text if text and isinstance(text, str) else "" for text in examples['Document']]
    targets = [text if text and isinstance(text, str) else "" for text in examples['Summary']]
    
    # Tokenize inputs with padding and truncation
    model_inputs = tokenizer(
        inputs, 
        max_length=1024, 
        truncation=True,
        padding="max_length",
    )
    
    # Tokenize targets with padding and truncation
    labels = tokenizer(
        targets, 
        max_length=150, 
        truncation=True,
        padding="max_length",
    )
    
    model_inputs["labels"] = labels["input_ids"]
    
    # Replace padding token id with -100 in labels so it's ignored in loss calculation
    model_inputs["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in label] 
        for label in model_inputs["labels"]
    ]
    
    return model_inputs

# Tokenize the dataset and remove the original columns
try:
    print("Tokenizing dataset...")
    tokenized_dataset = dataset.map(
        tokenize_function, 
        batched=True, 
        batch_size=16,  # Process in smaller batches to avoid memory issues
        remove_columns=dataset.column_names,
        desc="Tokenizing dataset"
    )
    print("Dataset tokenized successfully!")
except Exception as e:
    print(f"Error during tokenization: {e}")
    raise

# Split the tokenized dataset into train (80%) and test (20%) sets
try:
    train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
    tokenized_dataset = DatasetDict({
        "train": train_test_split["train"],
        "test": train_test_split["test"]
    })
    print(f"Dataset split into {len(tokenized_dataset['train'])} training and {len(tokenized_dataset['test'])} test examples")
except Exception as e:
    print(f"Error splitting dataset: {e}")
    raise

# Function to compute metrics for evaluation
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    # Decode predicted sequences
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 with pad token id and then decode
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Calculate ROUGE scores
    result = calculate_rouge_scores(decoded_preds, decoded_labels)
    
    # Convert scores to percentages
    result = {key: round(value * 100, 4) for key, value in result.items()}
    return result

# Create a data collator for dynamic padding
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8  # For better efficiency on tensors
)

# Define a progress callback to monitor training
class ProgressPercentageCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.max_steps and state.global_step:
            percent_complete = (state.global_step / state.max_steps) * 100
            print(f"Training progress: {percent_complete:.2f}% complete (step {state.global_step}/{state.max_steps})")
            
            # Monitor GPU usage if available
            if torch.cuda.is_available():
                try:
                    for i in range(torch.cuda.device_count()):
                        gpu_mem = torch.cuda.memory_allocated(i) / (1024**3)  # Convert to GB
                        print(f"GPU {i} Memory: {gpu_mem:.2f} GB")
                except Exception as e:
                    print(f"Error monitoring GPU: {e}")
        return control

# Set training parameters with memory management optimizations
# Reduce learning rate for further fine-tuning of an already fine-tuned model
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-5,  # Lower learning rate for further fine-tuning
    weight_decay=0.01,
    fp16=True,
    logging_dir=f"{output_dir}/logs",
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=200,
    metric_for_best_model="rouge1",
    load_best_model_at_end=True,
    greater_is_better=True,
    report_to="none"
)

print("Training arguments configured successfully!")

# Define the Trainer with evaluation metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Add the progress callback to monitor training progress
trainer.add_callback(ProgressPercentageCallback)

# Implement basic error handling for training
try:
    print("Starting further fine-tuning...")
    trainer.train()
    print("Further fine-tuning completed successfully!")
except Exception as e:
    print(f"Error during training: {e}")
    # If OOM error, suggest reducing batch size or model size
    if "CUDA out of memory" in str(e):
        print("Suggestion: Try reducing batch size, using gradient accumulation, or reducing model size")
    raise

# Save the final model after training
try:
    print("Saving final model...")
    trainer.save_model(final_model_path)
    print(f"Model saved to {final_model_path}")
    
    # Save LoRA configuration separately for future reference
    lora_config_path = os.path.join(final_model_path, "lora_config.json")
    lora_config.save_pretrained(final_model_path)
    print(f"LoRA configuration saved to {final_model_path}")
except Exception as e:
    print(f"Error saving model: {e}")
    raise

print("Further fine-tuning process completed!")

Loading custom fine-tuned model from: /kaggle/input/bartft/transformers/default/1


/usr/local/lib/python3.10/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(
Loading adapter weights from /kaggle/input/bartft/transformers/default/1 led to unexpected keys not found in the model: model.encoder.layers.0.self_attn.k_proj.lora_A.default.wei

Loading model in FP16...
Model loaded successfully on cuda!
Loaded model appears to be a PEFT model already
Target modules for LoRA: ['q_proj', 'k_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
trainable params: 8,650,752 || all params: 414,941,184 || trainable%: 2.0848
LoRA adapters applied successfully!
Loading custom dataset...
Dataset loaded successfully with 371 rows
Dataset columns: ['Paper Id', 'Paper Title', 'Key Words', 'Abstract', 'Conclusion', 'Document', 'Paper Type', 'Summary', 'Topic', 'OCR', 'labels']

Sample data:
Document    Multi-document Summarization via Deep Learning...
Summary     This article presents a systematic overview of...
Name: 0, dtype: object
Tokenizing dataset...


Tokenizing dataset:   0%|          | 0/371 [00:00<?, ? examples/s]

Dataset tokenized successfully!
Dataset split into 296 training and 75 test examples
Training arguments configured successfully!
Starting further fine-tuning...


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-33-ed87c207d645>:305: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss


Training progress: 100.00% complete (step 27/27)
GPU 0 Memory: 7.18 GB
GPU 1 Memory: 0.02 GB
Further fine-tuning completed successfully!
Saving final model...
Model saved to ./bart_further_lora_final_compscholar
LoRA configuration saved to ./bart_further_lora_final_compscholar
Further fine-tuning process completed!
